<a href="https://colab.research.google.com/github/cybersipra/british-airways-internship/blob/main/scraping_and_analysis/scraping_and_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup


In [124]:
import nltk
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [125]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [114]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [69]:
#create an empty list to collect all reviews
reviews  = []

#create an empty list to collect rating stars
stars = []

#create an empty list to collect date
date = []

#create an empty list to collect country the reviewer is from
country = []

In [70]:
for i in range(1, 36):
    page = requests.get(f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100")
    
    soup = BeautifulSoup(page.content, "html5")
    
    for item in soup.find_all("div", class_="text_content"):
        reviews.append(item.text)
    
    for item in soup.find_all("div", class_ = "rating-10"):
        try:
            stars.append(item.span.text)
        except:
            print(f"Error on page {i}")
            stars.append("None")
            
    #date
    for item in soup.find_all("time"):
        date.append(item.text)
        
    #country
    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.text.strip(" ()"))

Error on page 30
Error on page 31
Error on page 31
Error on page 34
Error on page 34


In [97]:
reviews = reviews[:3418]
country = country[:3418]
stars = stars[:3418]
date = date[:3418]
len(date)

3418

In [99]:
df = pd.DataFrame({"reviews":reviews,"stars": stars, "date":date, "country": country})


In [100]:
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | I will never fly British Ai...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,13th April 2023,United States
1,✅ Trip Verified | Worst aircraft I have ever ...,2,12th April 2023,United Kingdom
2,✅ Trip Verified | I enjoyed my flight. The bo...,1,11th April 2023,United Kingdom
3,✅ Trip Verified | Why do you make it so hard? ...,9,10th April 2023,United Kingdom
4,✅ Trip Verified | After several delays and ca...,5,9th April 2023,United States


In [104]:
df.to_csv("data/BA_reviews.csv")

In [108]:
df = pd.read_csv("data/BA_reviews.csv", index_col=0, skipinitialspace = True)


,reviews,stars,date,country
0,✅ Trip Verified | I will never fly British Ai...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,13th April 2023,United States
1,✅ Trip Verified | Worst aircraft I have ever ...,2,12th April 2023,United Kingdom
2,✅ Trip Verified | I enjoyed my flight. The bo...,1,11th April 2023,United Kingdom
3,✅ Trip Verified | Why do you make it so hard? ...,9,10th April 2023,United Kingdom
4,✅ Trip Verified | After several delays and ca...,5,9th April 2023,United States


In [111]:
df.head()

,reviews,stars,date,country,verified
0,✅ Trip Verified | I will never fly British Ai...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,13th April 2023,United States,True
1,✅ Trip Verified | Worst aircraft I have ever ...,2,12th April 2023,United Kingdom,True
2,✅ Trip Verified | I enjoyed my flight. The bo...,1,11th April 2023,United Kingdom,True
3,✅ Trip Verified | Why do you make it so hard? ...,9,10th April 2023,United Kingdom,True
4,✅ Trip Verified | After several delays and ca...,5,9th April 2023,United States,True


In [109]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [110]:
df['verified']


0        True
1        True
2        True
3        True
4        True
        ...  
3413    False
3414    False
3415    False
3416    False
3417    False
Name: verified, Length: 3418, dtype: bool

In [126]:
#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()


reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [127]:
df['corpus'] = corpus

In [128]:
df.head()


,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | I will never fly British Ai...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,13th April 2023,United States,True,never fly british airway start plane hour late...
1,✅ Trip Verified | Worst aircraft I have ever ...,2,12th April 2023,United Kingdom,True,worst aircraft ever flown seat cramped uncomfo...
2,✅ Trip Verified | I enjoyed my flight. The bo...,1,11th April 2023,United Kingdom,True,enjoyed flight boarding swift service friendly...
3,✅ Trip Verified | Why do you make it so hard? ...,9,10th April 2023,United Kingdom,True,make hard lounge experience staff taking care ...
4,✅ Trip Verified | After several delays and ca...,5,9th April 2023,United States,True,several delay canceled flight finally made cop...


In [129]:
df.dtypes

reviews     object
stars       object
date        object
country     object
verified      bool
corpus      object
dtype: object

In [130]:
df.date = pd.to_datetime(df.date)

In [131]:
df.date.head()


0   2023-04-13
1   2023-04-12
2   2023-04-11
3   2023-04-10
4   2023-04-09
Name: date, dtype: datetime64[ns]

In [133]:
df.stars.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '2', '1', '9', '5', '7', '4', '3',
       '10', '8', '6', 'None'], dtype=object)

In [134]:
df.stars = df.stars.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [135]:
df.stars.value_counts()


1       759
2       387
3       386
8       346
7       296
10      294
9       292
5       247
4       233
6       173
None      5
Name: stars, dtype: int64

In [137]:
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)

In [138]:
df.stars.unique()

array(['5', '2', '1', '9', '7', '4', '3', '10', '8', '6'], dtype=object)

In [141]:
df.isnull().value_counts()


reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     3411
                       True     False     False        2
dtype: int64

In [150]:
df.country.isnull().value_counts()

False    3411
Name: country, dtype: int64

In [144]:
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [145]:
df.shape

(3411, 6)

In [146]:
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | I will never fly British Ai...,5,2023-04-13,United States,True,never fly british airway start plane hour late...
1,✅ Trip Verified | Worst aircraft I have ever ...,2,2023-04-12,United Kingdom,True,worst aircraft ever flown seat cramped uncomfo...
2,✅ Trip Verified | I enjoyed my flight. The bo...,1,2023-04-11,United Kingdom,True,enjoyed flight boarding swift service friendly...
3,✅ Trip Verified | Why do you make it so hard? ...,9,2023-04-10,United Kingdom,True,make hard lounge experience staff taking care ...
4,✅ Trip Verified | After several delays and ca...,5,2023-04-09,United States,True,several delay canceled flight finally made cop...
...,...,...,...,...,...,...
3406,Business class from Shanghai to London. It was...,4,2014-06-16,United Kingdom,False,business class shanghai london worst business ...
3407,Paphos - LGW 11th June. We had monitored the o...,2,2014-06-16,United Kingdom,False,paphos lgw th june monitored outbound flight i...
3408,Flew from Edinburgh to New York via LHR on 5th...,7,2014-06-13,United Kingdom,False,flew edinburgh new york via lhr th june flight...
3409,Outbound BA 175 12/5/14. Return BA 286 1/6/14....,6,2014-06-13,United Kingdom,False,outbound ba return ba recently travelled econo...


In [147]:
df.to_csv("data/cleaned-BA-reviews.csv")


In [4]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,✅ Trip Verified | I will never fly British Ai...
1,✅ Trip Verified | Worst aircraft I have ever ...
2,✅ Trip Verified | I enjoyed my flight. The bo...
3,✅ Trip Verified | Why do you make it so hard? ...
4,✅ Trip Verified | After several delays and ca...


In [5]:
df.to_csv("ba internship/BA_reviews.csv")

In [28]:
ba_data = pd.read_csv("ba internship/BA_reviews.csv")
print(ba_data.head())

   Unnamed: 0                                            reviews
0           0  ✅ Trip Verified |  I will never fly British Ai...
1           1  ✅ Trip Verified |  Worst aircraft I have ever ...
2           2  ✅ Trip Verified |  I enjoyed my flight. The bo...
3           3  ✅ Trip Verified | Why do you make it so hard? ...
4           4  ✅ Trip Verified |  After several delays and ca...


In [29]:
ba_data['reviews'] = ba_data['reviews'].str.replace('✅ Trip Verified |', '')

<ipython-input-29-310106c9a2ee>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ba_data['reviews'] = ba_data['reviews'].str.replace('✅ Trip Verified |', '')


In [50]:
ba_data['reviews'] = ba_data['reviews'].str.replace('|', '')
ba_data['reviews'] = ba_data['reviews'].str.replace('Not Verified', '')
ba_data['reviews'] = ba_data['reviews'].str.replace('\r\n', '')


<ipython-input-50-92a3b1be22bd>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ba_data['reviews'] = ba_data['reviews'].str.replace('|', '')


In [44]:
y = ba_data[ba_data['reviews'].str.contains('Not Verified')]
print (y)

Empty DataFrame
Columns: [Unnamed: 0, reviews]
Index: []


In [51]:
y = ba_data[ba_data['reviews'].str.contains('\r\n')]
print (y)

Empty DataFrame
Columns: [Unnamed: 0, reviews]
Index: []


In [66]:
ba_data.to_csv("ba internship/BA_reviews.csv")

In [67]:
ba_data = pd.read_csv("ba internship/BA_reviews.csv", skipinitialspace = True)

In [68]:
ba_data.tail()


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,reviews
995,995,995,995,995,995,Flew on a last minute emergency trip London ...
996,996,996,996,996,996,"Earlier this year, my wife and I travelled b..."
997,997,997,997,997,997,"Los Angeles to London. I booked with AA, LA..."
998,998,998,998,998,998,The overall flight wasn't too bad. Seats an...
999,999,999,999,999,999,BA’s Galleries lounge at CPT is adequate but ...


# New Section

# New Section

In [8]:
ba_data.tail()

,Unnamed: 0,reviews
995,995,✅ Trip Verified | Flew on a last minute emerg...
996,996,"✅ Trip Verified | Earlier this year, my wife ..."
997,997,Not Verified | Los Angeles to London. I booke...
998,998,Not Verified | The overall flight wasn't too ...
999,999,✅ Trip Verified | \r\nBA’s Galleries lounge at...


In [9]:
ba_data.isnull()

,Unnamed: 0,reviews
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
995,False,False
996,False,False
997,False,False
998,False,False


In [10]:
ba_data.isna().any().sum()
 

0

In [11]:
ba_data.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [17]:
ba_data['reviews'] = ba_data['reviews'].str.replace('NotVerified |', '')

<ipython-input-17-1fe7596e0eae>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ba_data['reviews'] = ba_data['reviews'].str.replace('NotVerified |', '')


In [18]:
ba_data['reviews']

0      |IwillneverflyBritishAirwaysagain.Tostart,thep...
1      |WorstaircraftIhaveeverflown.Theseatswerecramp...
2      |Ienjoyedmyflight.Theboardingwasswiftandservic...
3      |Whydoyoumakeitsohard?Afterasosoloungeexperien...
4      |Afterseveraldelaysandcanceledflights,wefinall...
                             ...                        
995    |FlewonalastminuteemergencytripLondonHeathrowt...
996    |Earlierthisyear,mywifeandItravelledbusinesscl...
997    NotVerified|LosAngelestoLondon.IbookedwithAA,L...
998    NotVerified|Theoverallflightwasn'ttoobad.Seats...
999    |\r\nBA’sGalleriesloungeatCPTisadequatebutletd...
Name: reviews, Length: 1000, dtype: object

In [25]:
ba_data['reviews'] = ba_data['reviews'].str.replace('|', '')

<ipython-input-25-3fd0f2dd534e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ba_data['reviews'] = ba_data['reviews'].str.replace('|', '')


In [27]:
ba_data.head()

,Unnamed: 0,reviews
0,0,"IwillneverflyBritishAirwaysagain.Tostart,thepl..."
1,1,WorstaircraftIhaveeverflown.Theseatswerecrampe...
2,2,Ienjoyedmyflight.Theboardingwasswiftandservice...
3,3,Whydoyoumakeitsohard?Afterasosoloungeexperienc...
4,4,"Afterseveraldelaysandcanceledflights,wefinally..."
